<a href="https://www.kaggle.com/code/vtrackstar/dq-labs-retail-sales-performance-report-project?scriptVersionId=212716452" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Ask
* **Goals**
1. Overall performanc of DQLab Store from 2009 until 2012 for Order Numbers and Total Sales which order status is finished.
2. Overall performance of DQLab through product subcategory to be in contrast between 2011 and 2012.
3. The effectiveness and effectivity of promotions carried out so far, by means of calculating the brun rate of the overall promotions by way of year.
4. The effectiveness and effectivity of promotions carried out so far, via calculating the burn rate of the overall promotions by way of subcategory of product on 2012.
5. Analysis of clients each year.
6. Analysis range of new clients for every year. year
Analysis range of new clients for every. year year

## Prepare

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sales-performance-report-dqlab-store/clean_data.csv
/kaggle/input/sales-performance-report-dqlab-store/raw_data.csv


In [2]:
dqlab = pd.read_csv('/kaggle/input/sales-performance-report-dqlab-store/clean_data.csv')
dqlab

,order_id,order_status,customer,order_date,order_quantity,sales,discount,discount_value,product_category,product_sub_category
0,3,Order Finished,Muhammed Mac Intyre,2010-10-13,6,523080,0.04,20923,Office Supplies,Storage & Organization
1,293,Order Finished,Barry French,2012-10-01,49,20246040,0.07,1417223,Office Supplies,Appliances
2,483,Order Finished,Clay Rozendal,2011-07-10,30,9931519,0.08,794522,Technology,Telephones & Communication
3,515,Order Finished,Carlos Soltero,2010-08-28,19,788540,0.08,63083,Office Supplies,Appliances
4,613,Order Finished,Carl Jackson,2011-06-17,12,187080,0.03,5612,Office Supplies,Binders & Binder Accessories
...,...,...,...,...,...,...,...,...,...,...
5494,47360,Order Finished,Philip Brown,2010-10-08,25,4401280,0.05,220064,Furniture,Chairs & Chairmats
5495,52706,Order Finished,Paul Prost,2012-07-09,34,2083320,0.02,41666,Office Supplies,Binders & Binder Accessories
5496,54279,Order Finished,Lisa De Cherney,2011-07-30,41,20142180,0.10,2014218,Technology,Office Machines
5497,55558,Order Finished,Philip Brown,2010-08-09,8,2588080,0.05,129404,Furniture,Bookcases


In [3]:
dqlab.describe()

,order_id,order_quantity,sales,discount,discount_value
count,5499.000000,5499.000000,5.499000e+03,5499.000000,5.499000e+03
mean,29970.202219,25.521549,3.532838e+06,0.049915,1.735048e+05
std,17243.318085,14.485352,7.305121e+06,0.031783,4.183615e+05
min,3.000000,1.000000,6.460000e+03,0.000000,0.000000e+00
25%,15044.500000,13.000000,2.826700e+05,0.020000,7.739000e+03
50%,29927.000000,26.000000,8.546400e+05,0.050000,3.191700e+04
75%,44646.500000,38.000000,3.298741e+06,0.080000,1.329000e+05
max,59973.000000,50.000000,1.781221e+08,0.170000,7.441778e+06


In [4]:
dqlab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5499 entries, 0 to 5498
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   order_id              5499 non-null   int64  
 1   order_status          5499 non-null   object 
 2   customer              5499 non-null   object 
 3   order_date            5499 non-null   object 
 4   order_quantity        5499 non-null   int64  
 5   sales                 5499 non-null   int64  
 6   discount              5499 non-null   float64
 7   discount_value        5499 non-null   int64  
 8   product_category      5499 non-null   object 
 9   product_sub_category  5499 non-null   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 429.7+ KB


## Process and Analyze

In [5]:
## Overall Performance By Years
dqlab['order_date'] = pd.to_datetime(dqlab['order_date'])
finorder = dqlab[dqlab['order_status'] == 'Order Finished']

perfyr = (
    finorder.groupby(finorder['order_date'].dt.year)
    .agg(sales=('sales', 'sum'), number_of_orders=('order_status', 'count'))
    .reset_index().rename(columns={'order_date':'year'})
)
perfyr.head()

,year,sales,number_of_orders
0,2009,4613872681,1244
1,2010,4059100607,1248
2,2011,4112036186,1178
3,2012,4482983158,1254


In [6]:
## Overall Performance by Sub Category
filtered = dqlab[
    (dqlab['order_date'].dt.year.between(2011, 2012)) &
    (dqlab['order_status'] == 'Order Finished')
]

bestperf = (
    filtered.groupby([filtered['order_date'].dt.year, 'product_sub_category'])
    .agg(sales=('sales', 'sum'))
    .reset_index().rename(columns={'order_date': 'year'})
)

sortedsub = bestperf.sort_values(by=['year', 'sales'], ascending=[True, False])
sortedsub

,year,product_sub_category,sales
3,2011,Chairs & Chairmats,622962720
9,2011,Office Machines,545856280
15,2011,Tables,505875008
5,2011,Copiers & Fax,404074080
16,2011,Telephones & Communication,392194658
1,2011,Binders & Binder Accessories,298023200
14,2011,Storage & Organization,285991820
0,2011,Appliances,272630020
4,2011,Computer Peripherals,232677960
2,2011,Bookcases,169304620


In [7]:
filtered10 = dqlab[dqlab['order_status'] == 'Order Finished']
filtered1 = filtered10.copy()
filtered1['2011Sales'] = np.where(filtered1['order_date'].dt.year == 2011, filtered1['sales'], 0)
filtered1['2012Sales'] = np.where(filtered1['order_date'].dt.year == 2012, filtered1['sales'], 0)

grouped = (
    filtered1.groupby('product_sub_category')
    .agg(
        Sales2011=('2011Sales', 'sum'), Sales2012=('2012Sales', 'sum')
    ).reset_index()
)

grouped['sales_growth%'] = np.round(
    ((grouped['Sales2012'] - grouped['Sales2011']) * 100 / grouped['Sales2012']), 1)

sorted_sg = grouped.sort_values(by='sales_growth%', ascending=False)
sorted_sg

,product_sub_category,Sales2011,Sales2012,sales_growth%
13,"Scissors, Rulers & Trimmers",12638340,36776400,65.6
6,Envelopes,36463900,58629280,37.8
9,Office Machines,545856280,811427140,32.7
4,Computer Peripherals,232677960,308014340,24.5
14,Storage & Organization,285991820,356714140,19.8
12,Rubber Bands,3090120,3837880,19.5
1,Binders & Binder Accessories,298023200,363879200,18.1
10,Paper,111080380,126896160,12.5
8,Office Furnishings,160471500,178927480,10.3
16,Telephones & Communication,392194658,422287514,7.1


In [8]:
## Promotion Effectiveness and Efficiency by Year

filtered2 = filtered10.copy()
filtered2['year'] = filtered2['order_date'].dt.year

grouped1 = (
    filtered2.groupby('year')
    .agg(sales=('sales', 'sum'), promotion_value=('discount_value', 'sum')
    ).reset_index()
)

grouped1['burn_rate%'] = np.round(
    ((grouped1['promotion_value'] * 100) / grouped1['sales']), 2)
grouped1

,year,sales,promotion_value,burn_rate%
0,2009,4613872681,214330327,4.65
1,2010,4059100607,197506939,4.87
2,2011,4112036186,214611556,5.22
3,2012,4482983158,225867642,5.04


In [9]:
## Promotion Effectiveness and Efficiency by Product Sub Category

filtered0 = dqlab[dqlab['order_status'] == 'Order Finished']
filtered3 = filtered0.copy()

filtered3['year'] = filtered3['order_date'].dt.year
filtered3 = filtered3[filtered3['year'] == 2012]

grouped2 = (
    filtered3.groupby(['product_category', 'product_sub_category', dqlab['order_date'].dt.year])
    .agg(sales=('sales', 'sum'), promotion_value=('discount_value', 'sum')
        ).reset_index().rename(columns={'order_date':'year'})
)
promo1 = grouped2.sort_values(by='sales', ascending=False)
promo1

,product_category,product_sub_category,year,sales,promotion_value
15,Technology,Office Machines,2012,811427140,46616695
1,Furniture,Chairs & Chairmats,2012,654168740,26623882
16,Technology,Telephones & Communication,2012,422287514,18800188
3,Furniture,Tables,2012,388993784,16348689
5,Office Supplies,Binders & Binder Accessories,2012,363879200,22338980
12,Office Supplies,Storage & Organization,2012,356714140,18802166
13,Technology,Computer Peripherals,2012,308014340,15333293
14,Technology,Copiers & Fax,2012,292489800,14530870
4,Office Supplies,Appliances,2012,266131100,14393300
2,Furniture,Office Furnishings,2012,178927480,8233849


In [10]:
## Customer Transactions Per Year
filtered4 = filtered0.copy()
filtered4['year'] = filtered4['order_date'].dt.year

customer = (
    filtered4.groupby('year')
    .agg(number_of_customers=('customer', 'nunique')).reset_index()
)

customer

,year,number_of_customers
0,2009,585
1,2010,593
2,2011,581
3,2012,594


In [11]:
## New Customers Over The Years

filtered5 = filtered0.copy()

first = (
    filtered5.groupby('customer')
    .agg(first_order=('order_date', 'min'))
    .reset_index()
)

first['year'] = first['first_order'].dt.year

new_cust = (
    first.groupby('year')
    .agg(new_customers=('customer', 'count'))
    .reset_index()
)

new_cust

,year,new_customers
0,2009,585
1,2010,141
2,2011,38
3,2012,11


In [12]:
dqlab.to_csv('dqlabsalesperformancedata.csv', index=False)
sorted_sg.to_csv('dqlabsalesgrowth.csv', index=False)
grouped1.to_csv('dqlabburnrate.csv', index=False)
promo1.to_csv('dqlabpromoresult.csv', index=False)
new_cust.to_csv('dqlabnewcustomers.csv', index=False)

## Share

In [13]:
print("1. Overall Performance by Years")
print(perfyr.head())
print("2. Overall Performance by Sub Category")
print(sortedsub.head(30))
print(sorted_sg.head(30))
print("3. Promotion Effectiveness and Efficiency by Year")
print(grouped1.head())
print("4. Promotion Effectiveness and Efficiency by Product Sub Category")
print(promo1.head(30))
print("5. Customers Transactions Per Year")
print(customer.head())
print("6. New Customers Over The Years")
print(new_cust.head())

1. Overall Performance by Years
   year       sales  number_of_orders
0  2009  4613872681              1244
1  2010  4059100607              1248
2  2011  4112036186              1178
3  2012  4482983158              1254
2. Overall Performance by Sub Category
    year          product_sub_category      sales
3   2011            Chairs & Chairmats  622962720
9   2011               Office Machines  545856280
15  2011                        Tables  505875008
5   2011                 Copiers & Fax  404074080
16  2011    Telephones & Communication  392194658
1   2011  Binders & Binder Accessories  298023200
14  2011        Storage & Organization  285991820
0   2011                    Appliances  272630020
4   2011          Computer Peripherals  232677960
2   2011                     Bookcases  169304620
8   2011            Office Furnishings  160471500
10  2011                         Paper  111080380
11  2011           Pens & Art Supplies   43093800
6   2011                     Envelopes 

[View Interactive Tableau Dashboard](https://public.tableau.com/views/DQLabsSalesDashboardProject/DQLabSalesPerformanceDashboard)


<div class='tableauPlaceholder' id='viz1734016987006' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;DQ&#47;DQLabsSalesDashboardProject&#47;DQLabSalesPerformanceDashboard&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='DQLabsSalesDashboardProject&#47;DQLabSalesPerformanceDashboard' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;DQ&#47;DQLabsSalesDashboardProject&#47;DQLabSalesPerformanceDashboard&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1734016987006');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='850px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='850px';} else { vizElement.style.width='100%';vizElement.style.height='1050px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Final Insights
1. The total income and order volume for DQLab Store have fluctuated over the years. **The highest total income was recorded in 2009**. While the number of orders generally increased, **the growth was not significant**, particularly in 2011.
2. Total income by product sub-category showed an **overall increase between 2011 and 2012.** However, some sub-categories experienced a decline in income during this period, including **Appliances, Bookcases, Tables, Labels, and Copiers & Fax.**
3. The burn rate remained consistently **above 4.5% each year**, indicating that the effectiveness and efficiency of promotions did not meet DQLab Store's expectations.
4. Many products had a burn rate **exceeding 4.5%**, contributing to the overall high burn rate each year. Only a few products maintained a burn rate below 4.5%, including **Rubber Bands, Envelopes, Chairs & Chairmats, Tables, and Telephones & Communication.**
5. The number of customers remained **relatively stable** over the years, ranging between 580 and 590 annually.
6. The number of new customers has been **decreasing over time**, with the lowest count recorded in 2012 at just **11 new customers**.
2012.